#### First let's import a few things

In [1]:
import os
import ceci
from ceci.pipeline import Pipeline

#### For now we need to be in the top-level ceci directory to read the input yaml file

This is b/c it refers to the configuration file, which we will also need to read.

We should fix this.

In [2]:

os.chdir(os.getcwd() + '/../')
pipeline = Pipeline.read('tests/test.yml')

In [3]:
pipeline.stages

#### Now we have some pipeline introspection

1. Printing a list of stages
2. Accessing a stage as an attribute of the pipeline
3. Printing the inputs and output connections associated to a stage
4. Printing the configuration associated to a stage

In [4]:
pipeline.print_stages()

SysMapMaker         : <ceci_example.example_stages.SysMapMaker object at 0x1087ad690>
shearMeasurementPipe: <ceci_example.example_stages.shearMeasurementPipe object at 0x1087ad6f0>
PZEstimationPipe    : <ceci_example.example_stages.PZEstimationPipe object at 0x1087aca60>
WLGCRandoms         : <ceci_example.example_stages.WLGCRandoms object at 0x1087ac670>
WLGCSelector        : <ceci_example.example_stages.WLGCSelector object at 0x1087ac820>
SourceSummarizer    : <ceci_example.example_stages.SourceSummarizer object at 0x1087acf10>
WLGCTwoPoint        : <ceci_example.example_stages.WLGCTwoPoint object at 0x1087ae380>
WLGCCov             : <ceci_example.example_stages.WLGCCov object at 0x1087ac700>
WLGCSummaryStatistic: <ceci_example.example_stages.WLGCSummaryStatistic object at 0x1087ad450>


In [5]:
print(pipeline.SysMapMaker)

In [6]:
pipeline.SysMapMaker.print_io()

Inputs--------
DM                   : DM                   :<class 'ceci_example.types.TextFile'> : ./tests/inputs/dm.txt
Outputs--------
diagnostic_maps      : diagnostic_maps      :<class 'ceci_example.types.TextFile'> : diagnostic_maps.txt


In [7]:
print(repr(pipeline.WLGCSelector.config))

StageConfig{zbin_edges:[0.2, 0.3, 0.5],ra_range:[-5, 5],metacalibration:True,shear_catalog:./shear_catalog.txt,photoz_pdfs:./photoz_pdfs.txt,config:./tests/config.yml,}


#### Now we show how to save and read back a pipeline

The two different versions (i.e., with and without the reduce_config=True) will save slightly different versions
of the yaml file.  

In the former case we explicitly save everything for each stage of the pipeline.
In the later case, we reduce the configuration by parsing out the inputs, outputs and global params.  

In [8]:
pipeline.save('test_save.yml')

In [9]:
pipe_read = Pipeline.read('test_save.yml')

In [10]:
pipeline.save('test_save_v2.yml', reduce_config=True)

In [11]:
pipe_read_v2 = Pipeline.read('test_save_v2.yml')

#### Ok, eventually this example should include most stuff, but for now let's just the pipeline

In [12]:
status = pipeline.run()


Executing SysMapMaker
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example SysMapMaker   --DM=./tests/inputs/dm.txt   --config=./tests/config.yml   --diagnostic_maps=./tests/outputs/diagnostic_maps.txt 
Output writing to ./tests/logs/SysMapMaker.out

Job SysMapMaker has completed successfully in 0.5 seconds

Executing shearMeasurementPipe
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example shearMeasurementPipe   --DM=./tests/inputs/dm.txt   --config=./tests/config.yml   --shear_catalog=./tests/outputs/shear_catalog.txt 
Output writing to ./tests/logs/shearMeasurementPipe.out

Job shearMeasurementPipe has completed successfully in 0.5 seconds

Executing PZEstimationPipe
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example PZEstimationPipe   --DM=./tests/inputs/dm.txt   --fiducial_cosmology=./tests/inputs/fiducial_cosmology.txt   --config=./tests/config.yml   --photoz_pdfs=./tests/outputs/photoz_pdfs.txt 
Output writing to ./tests/logs/PZEstimationPipe.out

Job PZEstimationPip

#### Now let's do the same thing by building the pipeline with the python interface

In [13]:
from ceci_example.example_stages import *

In [14]:
pipe2 = Pipeline.interactive()
overall_inputs = {'DM':'./tests/inputs/dm.txt',
                  'fiducial_cosmology':'./tests/inputs/fiducial_cosmology.txt'}
inputs = overall_inputs.copy()
inputs['metacalibration'] = True
inputs['config'] = None

pipe2.pipeline_files.update(**inputs)
pipe2.build_stage(PZEstimationPipe)
pipe2.build_stage(shearMeasurementPipe, apply_flag=False)
pipe2.build_stage(WLGCSelector, zbin_edges=[0.2, 0.3, 0.5], ra_range=[-5, 5])
pipe2.build_stage(SysMapMaker)
pipe2.build_stage(SourceSummarizer)
pipe2.build_stage(WLGCCov, aliases={"covariance_1":"covariance_shared"})
pipe2.build_stage(WLGCRandoms)
pipe2.build_stage(WLGCTwoPoint)
pipe2.build_stage(WLGCSummaryStatistic, aliases={"covariance_2":"covariance_shared"})


{'wlgc_summary_data': './wlgc_summary_data.txt'}

In [15]:
print(pipe2.WLGCCov.config)

{DM:./tests/inputs/dm.txt,fiducial_cosmology:./tests/inputs/fiducial_cosmology.txt,metacalibration:True,config:None,photoz_pdfs:./photoz_pdfs.txt,shear_catalog:./shear_catalog.txt,tomography_catalog:./tomography_catalog.txt,diagnostic_maps:./diagnostic_maps.txt,source_summary_data:./source_summary_data.txt,}


In [16]:
run_config = {
    'output_dir': './tests/outputs',
    'log_dir': './tests/logs',
    'resume': 'restart',
}

stages_config = './tests/config.yml'

pipe2.initialize(overall_inputs, run_config,  stages_config);

In [17]:
pipe2.WLGCCov.outputs

[('covariance_1', ceci_example.types.TextFile)]

In [18]:
pipe2.WLGCCov.find_outputs(pipe2.run_config['output_dir'])

{'covariance_shared': './tests/outputs/covariance_shared.txt'}

In [19]:
pipe2.run()


Executing shearMeasurementPipe
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example shearMeasurementPipe   --DM=./tests/inputs/dm.txt   --config=./tests/config.yml   --shear_catalog=./tests/outputs/shear_catalog.txt 
Output writing to ./tests/logs/shearMeasurementPipe.out

Job shearMeasurementPipe has completed successfully in 0.5 seconds

Executing SysMapMaker
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example SysMapMaker   --DM=./tests/inputs/dm.txt   --config=./tests/config.yml   --diagnostic_maps=./tests/outputs/diagnostic_maps.txt 
Output writing to ./tests/logs/SysMapMaker.out

Job SysMapMaker has completed successfully in 0.5 seconds

Executing PZEstimationPipe
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example PZEstimationPipe   --DM=./tests/inputs/dm.txt   --fiducial_cosmology=./tests/inputs/fiducial_cosmology.txt   --config=./tests/config.yml   --photoz_pdfs=./tests/outputs/photoz_pdfs.txt 
Output writing to ./tests/logs/PZEstimationPipe.out

Job PZEstimationPip

0